In [116]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from keras import Sequential
from keras.src.callbacks import ModelCheckpoint
from keras.src.engine.input_layer import Input
from keras.src.layers import Dense, SimpleRNN
from keras.src.optimizers import Adam

In [164]:
data = np.sin(0.1 * np.arange(200)) - np.random.randn(200) / 10

px.line(data)

In [118]:
T = 20  # preview values using to predict
X = []
Y = []

for t in range(len(data) - T):
    x = data[t:t + T]
    y = data[t + T]
    X.append(x)
    Y.append(y)

X = np.array(X).reshape(-1, T, 1)
Y = np.array(Y)
N = len(X)
print(f"X shape: {X.shape}\nY shape: {Y.shape}")

X shape: (190, 10, 1)
Y shape: (190,)


In [174]:
model = Sequential()
model.add(Input(shape=(T, 1)))
model.add(SimpleRNN(units=5, activation="relu"))
model.add(Dense(units=1))
model.summary()

mc = ModelCheckpoint("best_model.keras", save_best_only=True, monitor="loss", mode="min", verbose=1)

model.compile(optimizer=Adam(learning_rate=0.1), loss="mse")

r = model.fit(X[:-N // 2], Y[:-N // 2], epochs=150, validation_data=(X[-N // 2:], Y[-N // 2:]),
              callbacks=[mc])

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_24 (SimpleRNN)   (None, 5)                 35        
                                                                 
 dense_33 (Dense)            (None, 1)                 6         
                                                                 
Total params: 41 (164.00 Byte)
Trainable params: 41 (164.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/150
1/3 [=========>....................] - ETA: 3s - loss: 0.1557
Epoch 1: loss improved from inf to 0.08604, saving model to best_model.keras
3/3 [==============================] - 2s 286ms/step - loss: 0.0860 - val_loss: 0.0633
Epoch 2/150
1/3 [=========>....................] - ETA: 0s - loss: 0.0495
Epoch 2: loss improved from 0.08604 to 0.02241, saving model to best_model.keras
3/3 [===============

In [175]:
def draw_history(r):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=r.epoch, y=r.history['loss'], name="loss"))
    fig.add_trace(go.Scatter(x=r.epoch, y=r.history['val_loss'], name="val_loss"))
    fig.show()


draw_history(r)

In [178]:
model.load_weights("best_model.keras")
x = X[-N // 2]
knowed = Y[-N // 2:]
predicted = []

for i in range(95):
    p = model.predict(x.reshape(1, -1, 1), verbose=0)[0, 0]
    predicted.append(p)

    x = np.roll(predicted, -1)
    x[-1] = p

In [179]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=knowed, name="true"))
fig.add_trace(go.Scatter(y=predicted, name="predicted"))
fig.show()